In [2]:
!pip install JPype1

In [5]:
!pip install konlpy

In [7]:
import konlpy

In [10]:
from konlpy.tag import Okt
tagger = Okt() # Twitter 대신 Okt 클래스 사용
tagger.pos('아버지가가방에들어가신다')

[('아버지', 'Noun'),
 ('가', 'Josa'),
 ('가방', 'Noun'),
 ('에', 'Josa'),
 ('들어가신다', 'Verb')]

In [12]:
tagger = Okt()
tagger.morphs('아버지가가방에들어가신다')

['아버지', '가', '가방', '에', '들어가신다']

In [15]:
tagger = Okt()
tagger.nouns('아버지가가방에들어가신다')

['아버지', '가방']

In [18]:
import pandas as pd

# 뉴스 기사 TDM
df = pd.read_csv('한국방송통신전파진흥원_전파누리_최신뉴스_20230725.csv',
encoding='cp949',
engine='python')

In [21]:
df.shape # 데이터 구성 확인

(1399, 4)

In [23]:
df.columns #칼럼 구성 확인

Index(['제목', '내용', '조회수', '출저'], dtype='object')

In [26]:
df['내용'].head() # 데이터 샘플 보기

0    <p><span>한국나노기술원(원장 서광석)은 국방&middot;의료&middot;...
1    <p><span>광주정보문화산업진흥원 컨소시엄은 의료&middot;헬스케어 AI융합...
2    <p><span>유영상 SK텔레콤 대표&middot;구현모 KT 대표&middot;...
3    <p><span>과학기술정보통신부와 정보통신산업진흥원(NIPA)이 주관하는 '인공지...
4    <p><span>과학기술정보통신부와 정보통신산업진흥원(NIPA)이 주관하는 'AI융...
Name: 내용, dtype: object

In [29]:
from konlpy.tag import Komoran  # 형태소 분석기 불러오기
tagger = Komoran()  # Komoran 형태소 분석기를 불러 tagger에 저장

def get_nouns(text):
    nouns = tagger.nouns(text)  # 형태소 분석기로 명사 추출
    nouns = [word for word in nouns if len(word) > 1]  # 한 글자 명사 제거
    return nouns

In [34]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=1000, tokenizer=get_nouns)

In [37]:
tdm = cv.fit_transform(df['내용'])
cv.get_feature_names_out()[:10]  # 첫 10 단어

array(['.com', '10대', '10월', '11월', '12월', '17', '19', '1월', '2019년',
       '2021년'], dtype=object)

In [41]:
from sklearn.feature_extraction.text import TfidfTransformer

# TF-IDF 변환기 초기화
tfidf = TfidfTransformer()

# TDM에 TF-IDF 적용
tdm2 = tfidf.fit_transform(tdm)

# 단어 목록 추출
words = cv.get_feature_names_out()

# 원래 TDM에서 각 단어와 빈도 출력
for i, n in sorted(zip(tdm[0].indices, tdm[0].data)):
    print(words[i], n)

# TF-IDF 변환된 TDM에서 각 단어와 가중치 출력
for i, n in sorted(zip(tdm2[0].indices, tdm2[0].data)):
    print(words[i], n)

구축 1
국방 1
국산 1
반도체 1
소재 1
양자 1
원장 1
의료 1
통신 1
플랫폼 1
한국 1
활용 1
구축 0.17306621812525108
국방 0.3551210310603936
국산 0.31944528622744156
반도체 0.3551210310603936
소재 0.36556679848655604
양자 0.2931921381374358
원장 0.35048711478649247
의료 0.32818449633429586
통신 0.0991047166686797
플랫폼 0.27215277991885245
한국 0.22509510104805375
활용 0.1762873210468014


In [43]:
import joblib  # joblib을 직접 임포트

# 데이터 저장
with open('ai_news.pkl', 'wb') as f:
    joblib.dump({'vectorizer': cv, 'tdm': tdm}, f)  # 들여쓰기 추가